# Guided Project: Answering Business Questions Using SQL

The aim of this guided project is to answer some business questions while practicing using SQL skills.  We'll use SQLite to query the Chinook database.

## Creating Helper Functions

In [1]:
import sqlite3
import numpy as np
import pandas as pd

def run_query(q):
    '''
    takes a SQL query (q) as an argument and returns a pandas dataframe of that query
    '''
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(q, conn)
    
def run_command(c):
    '''
    takes a SQL command (c) as an argument and executes it using the sqlite module.
    '''
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None    #to autocommit any changes
        conn.execute(c)

def show_tables():
    '''
    calls the run_query() function to return a list of all tables and views in the database.
    '''
    q = ''' 
         SELECT
             name,
             type
           FROM sqlite_master
          WHERE type IN ("table","view");
       '''
    return run_query(q)
            
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


## Selecting Albums to Purchase

The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:
* Regal: Hip-Hop
* Red Tone: Punk
* Meteor and the Girls: Pop
* Slim Jim Bites: Blues

We'll need to write a query to find out which genres sell the most tracks in the USA.

In [2]:
albums_to_purchase = '''
    WITH 
    
    tracks_usa AS (
        SELECT *
          FROM invoice_line il
          JOIN invoice i ON il.invoice_id = i.invoice_id
          JOIN customer c ON c.customer_id = i.customer_id
         WHERE c.country = "USA"
    )
    
    SELECT 
        g.name genre, 
        COUNT(*) tracks_sold,
        CAST(COUNT(*) AS float) / (SELECT COUNT(*)
                                     FROM tracks_usa) percentage_sold
      FROM genre g
      JOIN track t ON g.genre_id = t.genre_id
      JOIN tracks_usa tu ON tu.track_id = t.track_id
     GROUP BY genre
     ORDER BY tracks_sold DESC;
      
    
'''
run_query(albums_to_purchase)

,genre,tracks_sold,percentage_sold
0,Rock,561,0.533777
1,Alternative & Punk,130,0.123692
2,Metal,124,0.117983
3,R&B/Soul,53,0.050428
4,Blues,36,0.034253
5,Alternative,35,0.033302
6,Pop,22,0.020932
7,Latin,22,0.020932
8,Hip Hop/Rap,20,0.019029
9,Jazz,14,0.013321


Rock seems to be far more popular than any other genre, contributing to 53% of total sales. Unfortunately, we weren't offered rock albums.
Out of the 4 genres we are interested in (Hip-Hop, Punk, Pop, Blues) Alternative/Punk is the most popular, ranking second after Rock, so it's definitely an album to be purchased.

Blues contributes for 3% of the sales, Pop around 2%, Hip-Hop is the lowest with 1.9%, so the albums were should purchase based on the sales by genre are from the following artists:

* Red Tone: Punk
* Meteor and the Girls: Pop
* Slim Jim Bites: Blues

## Analysing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [3]:
sales_of_agents = '''
    WITH sales_by_agent_id AS (
            SELECT support_rep_id,
                   SUM(total) AS total_sales
              FROM customer c
              JOIN invoice i ON i.customer_id = c.customer_id
             GROUP BY support_rep_id
         )
    
    SELECT first_name || ' ' || last_name AS name,
           city || ', ' || country AS adress,
           hire_date,
           total_sales
      FROM employee e
      JOIN sales_by_agent_id sba ON sba.support_rep_id = e.employee_id
     WHERE title = "Sales Support Agent"
     ORDER BY total_sales DESC;
'''
run_query(sales_of_agents)

,name,adress,hire_date,total_sales
0,Jane Peacock,"Calgary, Canada",2017-04-01 00:00:00,1731.51
1,Margaret Park,"Calgary, Canada",2017-05-03 00:00:00,1584.00
2,Steve Johnson,"Calgary, Canada",2017-10-17 00:00:00,1393.92


Jean Peacock has made the highest sales so far, but she's been hired for the longest time, and the newest agent, Steve Johnson has made the lowest sales. From this data, the performance of the sales agents seems to be quite similar.

## Analysing Sales By Country

The next task is to analyze the sales data for customers from each different country. We have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, we have been directed to calculate data, for each country, on the:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

Because there are a number of countries with only one customer, these customers should be grouped as "Other" in our analysis.

In [4]:
sales_by_country = '''
    WITH
    is_other AS (
    
    SELECT
           CASE
             WHEN COUNT(DISTINCT c.customer_id) = 1 THEN "Other"
             ELSE c.country
             END AS country,
           COUNT(DISTINCT c.customer_id) customers,
           SUM(i.total) total_sales,
           SUM(i.total) / COUNT(DISTINCT c.customer_id) average_sales_per_customer,
           SUM(i.total) / COUNT(DISTINCT i.invoice_id) average_order_value
      FROM invoice i
      JOIN customer c ON c.customer_id = i.customer_id
      GROUP BY c.country
      ORDER BY total_sales DESC 
    )
    
    SELECT country,
           customers,
           total_sales, 
           average_sales_per_customer,
           average_order_value
      FROM (SELECT country,
                   SUM(customers) customers,
                   SUM(total_sales) total_sales, 
                   AVG(average_sales_per_customer) average_sales_per_customer,
                   AVG(average_order_value) average_order_value,
                   CASE
                    WHEN country = "Other" THEN 1
                    ELSE 0
                    END AS sort
              FROM is_other
             GROUP BY country
             ORDER BY sort, total_sales DESC
             )
'''
run_query(sales_by_country)

,country,customers,total_sales,average_sales_per_customer,average_order_value
0,USA,13,1040.49,80.037692,7.942672
1,Canada,8,535.59,66.948750,7.047237
2,Brazil,5,427.68,85.536000,7.011148
3,France,5,389.07,77.814000,7.781400
4,Germany,4,334.62,83.655000,8.161463
5,Czech Republic,2,273.24,136.620000,9.108000
6,United Kingdom,3,245.52,81.840000,8.768571
7,Portugal,2,185.13,92.565000,6.383793
8,India,2,183.15,91.575000,8.721429
9,Other,15,1094.94,72.996000,7.445071


Most of the customers and highest total sales are from the USA. Customers from Czech Republic, Portugal and India tend to spend more on average and the Czech Republic, United Kingdom  and India have the highest average orders.
The sample size is quite small making the result biased.